<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Version-history" data-toc-modified-id="Version-history-0.0.0.1"><span class="toc-item-num">0.0.0.1&nbsp;&nbsp;</span>Version history</a></span></li></ul></li></ul></li><li><span><a href="#Inference-and-Submission" data-toc-modified-id="Inference-and-Submission-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Inference and Submission</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Install-detectron-2" data-toc-modified-id="Install-detectron-2-0.1.0.1"><span class="toc-item-num">0.1.0.1&nbsp;&nbsp;</span>Install detectron 2</a></span></li><li><span><a href="#Input-dataset" data-toc-modified-id="Input-dataset-0.1.0.2"><span class="toc-item-num">0.1.0.2&nbsp;&nbsp;</span>Input dataset</a></span></li><li><span><a href="#Initiate-a-Predictor-from-the-trained-models" data-toc-modified-id="Initiate-a-Predictor-from-the-trained-models-0.1.0.3"><span class="toc-item-num">0.1.0.3&nbsp;&nbsp;</span>Initiate a Predictor from the trained models</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Inference" data-toc-modified-id="Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inference</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Create-the-submission.csv" data-toc-modified-id="Create-the-submission.csv-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>Create the submission.csv</a></span></li></ul></li></ul></li></ul></li></ul></div>

This notebook is based on Slawek Biel's notebook (https://www.kaggle.com/slawekbiel/positive-score-with-detectron-3-3-inference)

#### Version history
* V1 - test the model_best_5.pth
* V2 - test the model_best_4.pth
* V3 - test the model_final_6.pth
* V4 - test the model_final_5.pth

## Inference and Submission
After training the model ,we can inference with our trained models.

#### Install detectron 2

In [2]:
!pip install detectron

ERROR: Could not find a version that satisfies the requirement detectron (from versions: none)
ERROR: No matching distribution found for detectron


In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
import detectron2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fastcore.all import *

#### Input dataset

In [ ]:
dataDir=Path('../input/sartorius-cell-instance-segmentation')

In [ ]:
ids, masks=[],[]
test_names = (dataDir/'test').ls()

#### Initiate a Predictor from the trained models

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.INPUT.MASK_FORMAT='bitmask'
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
# cfg.MODEL.WEIGHTS = os.path.join('../input/detectron2-sartorius-cell-instance-segmentation', "model_final_11.pth")
# cfg.TEST.DETECTIONS_PER_IMAGE = 1000
# predictor = DefaultPredictor(cfg)
# THRESHOLDS = [.15, .35, .55]
# MIN_PIXELS = [75, 150, 75]

In [ ]:
weights = [
    "model_0009679.pth",
    "model_best_4.pth",
    "model_best_5.pth",
    "model_best_6.pth",
    "model_best_7.pth",
    "model_best_8.pth",
    "model_best_9.pth",
    "model_best_10.pth",
    "model_final_1.pth",
    "model_final_2.pth",
    "model_final_4.pth",
    "model_final_5.pth",
    "model_final_6.pth",
    "model_final_7.pth",
    "model_final_8.pth",
    "model_final_11.pth",
    "model_final_12.pth",
]

In [ ]:
#config_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
mdl_path = "../input/dtectron2-models-5fold"
DATA_PATH = "../input/sartorius-cell-instance-segmentation"
CONFIG = model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
MODELS = []
MODEL_NAMES =[]
THSS = []
ID_TEST = 0
SUBM_PATH = f'{DATA_PATH}/test'
SINGLE_MODE = False
NMS = True
IOU_TH = .4
THRESHOLDS = [.15, .35, .55]
MIN_PIXELS = [75, 150, 75]

for weight in weights:
    print(weight)
    model_name=weight[:-4]
    MODEL_NAMES.append(model_name)
    THSS.append(THRESHOLDS)
    cfg = get_cfg()
    cfg.merge_from_file(CONFIG)
    cfg.INPUT.MASK_FORMAT = 'bitmask'
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    cfg.MODEL.WEIGHTS = os.path.join('../input/detectron2sartoriuscellinstancesegmentation', weight)
    cfg.TEST.DETECTIONS_PER_IMAGE = 1000
    MODELS.append(DefaultPredictor(cfg))
print(f'all loaded:\nthresholds: {THSS}\nmodels: {BEST_MODELS}')

In [ ]:
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) 
                       for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def pred_masks(file_name, path, model, ths, min_pixels):
    img = cv2.imread(f'{path}/{file_name}')
    output = model(img)
    pred_classes = output['instances'].pred_classes.cpu().numpy().tolist()
    pred_class = max(set(pred_classes), key=pred_classes.count)
    take = output['instances'].scores >= ths[pred_class]
    pred_masks = output['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    result = []
    used = np.zeros(img.shape[:2], dtype=int) 
    for i, mask in enumerate(pred_masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

def ensemble_preds(file_name, path, models, ths):
    img = cv2.imread(f'{path}/{file_name}')
    classes = []
    scores = []
    bboxes = []
    masks = []
    for i, model in enumerate(models):
        output = model(img)
        pred_classes = output['instances'].pred_classes.cpu().numpy().tolist()
        pred_class = max(set(pred_classes), key=pred_classes.count)
        take = output['instances'].scores >= ths[i][pred_class]
        classes.extend(output['instances'].pred_classes[take].cpu().numpy().tolist())
        scores.extend(output['instances'].scores[take].cpu().numpy().tolist())
        bboxes.extend(output['instances'].pred_boxes[take].tensor.cpu().numpy().tolist())
        masks.extend(output['instances'].pred_masks[take].cpu().numpy())
    assert len(classes) == len(masks) , 'ensemble lenght mismatch'
    #scores, classes, bboxes, masks = zip(*sorted(zip(scores, classes, bboxes, masks),reverse=True))
    return classes, scores, bboxes, masks

def nms_predictions(classes, scores, bboxes, masks, 
                    iou_th=.5, shape=(520, 704)):
    he, wd = shape[0], shape[1]
    boxes_list = [[[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he] for x in bboxes]]
    scores_list = [[x for x in scores]]
    classes_list = [[x for x in classes]]
    nms_bboxes, nms_scores, nms_classes = non_maximum_weighted(
        boxes_list, 
        scores_list, 
        classes_list, 
        weights=None,
        iou_thr=0.3,skip_box_thr=0.0001  
    )
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
    nms_scores, nms_classes, nms_masks = zip(*sorted(zip(nms_scores, nms_classes, nms_masks), reverse=True))
    return nms_classes, nms_scores, nms_masks

def ensemble_pred_masks(masks, classes, min_pixels, shape=(520, 704)):
    result = []
    #pred_class = max(set(classes), key=classes.count)
    pred_class = int(max(set(classes), key=classes.count).item())
    used = np.zeros(shape, dtype=int) 
    for i, mask in enumerate(masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

In [ ]:
classes, scores, bboxes, masks = ensemble_preds(
    file_name=test_names[ID_TEST] , 
    path=SUBM_PATH, 
    models=MODELS, 
    ths=THSS
)
if NMS:
    classes, scores, masks = nms_predictions(
        classes, 
        scores, 
        bboxes,
        masks, iou_th=IOU_TH
    )
encoded_masks = ensemble_pred_masks(masks, classes, min_pixels=MIN_PIXELS)

In [ ]:
_, axs = plt.subplots(2, 2, figsize=(14, 8))
axs[0][0].imshow(cv2.imread(f'{SUBM_PATH}/{test_names[ID_TEST]}'))
axs[0][0].axis('off')
axs[0][0].set_title(test_names[ID_TEST])
for en_mask in encoded_masks_single:
    dec_mask = rle_decode(en_mask)
    axs[0][1].imshow(np.ma.masked_where(dec_mask == 0, dec_mask))
    axs[0][1].axis('off')
    axs[0][1].set_title('single model')
axs[1][0].imshow(cv2.imread(f'{SUBM_PATH}/{test_names[ID_TEST]}'))
axs[1][0].axis('off')
axs[1][0].set_title(test_names[ID_TEST])
for en_mask in encoded_masks:
    dec_mask = rle_decode(en_mask)
    axs[1][1].imshow(np.ma.masked_where(dec_mask == 0, dec_mask))
    axs[1][1].axis('off')
    axs[1][1].set_title('ensemble models')
plt.show()

# Inference

In [ ]:
subm_ids, subm_masks = [], []
for test_name in tqdm(test_names):
    if SINGLE_MODE:
        encoded_masks = pred_masks(
            test_name, 
            path=SUBM_PATH, 
            model=MODELS[0],
            ths=THSS[0],
            min_pixels=MIN_PIXELS
        )
    else:
        classes, scores, bboxes, masks = ensemble_preds(
            file_name=test_name, 
            path=SUBM_PATH, 
            models=MODELS, 
            ths=THSS
        )
        if NMS:
            classes, scores, masks = nms_predictions(
                classes, 
                scores, 
                bboxes, 
                masks, 
                iou_th=IOU_TH
            )
        encoded_masks = ensemble_pred_masks(
            masks, 
            classes, 
            min_pixels=MIN_PIXELS
        )
    for enc_mask in encoded_masks:
        subm_ids.append(test_name[:test_name.find('.')])
        subm_masks.append(enc_mask)

#### Create the submission.csv

In [ ]:
pd.DataFrame({
    'id': subm_ids, 
    'predicted': subm_masks
}).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

In [ ]:
# pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)
# pd.read_csv('submission.csv').head()